In [ ]:
import logging
import os
import urllib3

logging.captureWarnings(True)
http = urllib3.PoolManager()

repo_base = "https://raw.githubusercontent.com/earwole1/covid/master/"
files= ["covid.py", "population.py"]

for file in files:
  url = os.path.join(repo_base, file)
  r = http.request("GET", url)
  out_name = os.path.basename(url)
  with open(out_name, 'wb') as fp:
    fp.write(r.data)
    print(f"Copied {out_name}")


In [ ]:
from covid import Covid
c = Covid()

In [ ]:
# Find states with positive rate increasing faster than the testing rate over the last 28 days
worst_28 = c.find_worst_positive_to_testing(num_return=20, day_range=28)
c.print_worst_positive_to_testing(worst_28)

In [ ]:
_ = c.plot_all_by_date()

In [ ]:
from matplotlib import pyplot as plt

dates = c.data.date.unique()
states = c.data.state.unique()
hosp = []
data = c.data
for date in dates:
  temp = data[data.date == date]
  hosp_sum = temp.hospitalizedCurrently.sum()
  hosp.append(hosp_sum)
smoothed = c.running_average(hosp, 7)
fig, ax = plt.subplots()
fig.set_figwidth(11)
fig.set_figheight(6)
ax.grid()
ax.set_title("Covid Hospitalized - All States")
m = ax.plot(dates, hosp, 'bo', label='Hospitalized')
s = ax.plot(dates, smoothed, 'r-', lw=4, label='7-Day Average')
ax.legend()


In [ ]:
states = ["Ohio", "Maryland", "Connecticut", "Colorado"]
for state in states:
  _ = c.plot_state(state, "cases")

In [ ]:
def plot_worst_states(metric: str, num_to_plot: int):
  worst = c.find_worst_states(metric, num_to_plot, Covid.DEFAULT_DAYS)
  for w in worst:
    state = w[0]
    try:
      _ = c.plot_state(state, metric)
    except ValueError as VE:
      print(f"Missing data for {state}.")

  for w in worst:
    state = w[0]
    tup = w[2][0]
    x, y, m = tup
    N = len(x)
    print(f"{state:20} {metric} daily change increasing at {m:7.1f}/day over last {N} days")

In [ ]:
plot_worst_states("cases", 4)

In [ ]:
plot_worst_states("hospitalizedCurrently", 4)

In [ ]:
plot_worst_states("positive", 4)